In [ ]:
# nb_generic_bronze_to_silver
# Run config and functions

In [1]:
%run ./nb_config


StatementMeta(, 9ec994a7-7ca8-47dc-8e17-08a4477051b0, 3, Finished, Available, Finished)

Config Loaded Successfully


In [2]:
%run ./nb_logging


StatementMeta(, 9ec994a7-7ca8-47dc-8e17-08a4477051b0, 5, Finished, Available, Finished)

2026-01-04 17:43:25 - nb_logging - INFO - Logging notebook initialized successfully.


In [3]:
%run ./nb_functions

StatementMeta(, 9ec994a7-7ca8-47dc-8e17-08a4477051b0, 8, Finished, Available, Finished)

In [6]:
logger = get_logger(__name__)

dataframe_collection = {}
step_results = {}

# Block 2: Extract
current_stage = "extracted"
dataframe_collection[current_stage] = {}
total_count = 0

for source, configs in source_configs.items():
    src_path = f"{LKHS_PATH_SRC}{configs['source']['src_directory']}"
    if not is_directory_empty(src_path):
        df = spark.read.format(FILE_FORMAT).load(src_path)
        step_results[f"Extraction_{source}"] = "SUCCESS"
        count = df.count()
        if count > 0:
            dataframe_collection[current_stage][source] = df
            total_count += count

if total_count == 0:
    mssparkutils.notebook.exit("All source dataframes are empty.")

# Block 3a: Simplified
previous_stage = "extracted"
current_stage = "simplified"
dataframe_collection[current_stage] = {}

for source, data in dataframe_collection[previous_stage].items():
    configs = source_configs[source]
    if 'simplify_structure' in configs:
        for s in configs['simplify_structure']:
            if s["method"] == "explode": data = explode_col(data, s["col"])
            elif s["method"] == "flatten": data = flatten_struct_col(data, s["col"])
            elif s["method"] == "zip_explode": data = zip_explode_and_flatten(data, s["parent"], s["children"])
    dataframe_collection[current_stage][source] = data
step_results["Transformation_Simplified"] = "SUCCESS"

# Block 3b/c: Datatyped, Standardized & Normalized

dataframe_collection["standardized"] = {}

for k, v in dataframe_collection["simplified"].items():
    # 1. Run general cleaning (column names to snake_case)
    df_standard = clean_column_names_and_schemas(v)
    
    # 2. Specific Alignment for Energy: Rename to 'time'
    if k == "energy_config":
        if "readingdate" in df_standard.columns:
            df_standard = df_standard.withColumnRenamed("readingdate", "time")
        elif "reading_date" in df_standard.columns:
            df_standard = df_standard.withColumnRenamed("reading_date", "time")
            
    # 3. DATA TYPE NORMALIZATION: Force 'time' to Timestamp
    # This prevents the NULL join issue in Gold
    if "time" in df_standard.columns:
        df_standard = df_standard.withColumn("time", to_timestamp(col("time")))
            
    dataframe_collection["standardized"][k] = df_standard

step_results["Transformation_Standardized"] = "SUCCESS"
    
# Block 3d: Meta (Primary Key logic exactly same)
current_stage = "meta"
dataframe_collection[current_stage] = {}
for source, data in dataframe_collection["standardized"].items():
    configs = source_configs[source]
    if 'primary_col' in configs:
        pk_cfg = configs['primary_col']
        cols = [col(c).cast("string") for c in pk_cfg["lokaal_id_cols"]]
        data = data.withColumn("lokaal_id_col", concat_ws("_", *cols))
        data = data.withColumn(pk_cfg["primary_col_name"], hash_column(lit(pk_cfg["objecttype"]), lit(_SOURCE), col("lokaal_id_col")))
    
    data = data.withColumn(SOURCE_COL_NAME, lit(_SOURCE))
    data = data.withColumn(IMPORTDATE_COL_NAME, current_timestamp())
    dataframe_collection[current_stage][source] = data
step_results["Transformation_Meta"] = "SUCCESS"

# Block 3e: Selected
dataframe_collection["selected"] = {}
for source, data in dataframe_collection["meta"].items():
    if 'columns_to_select' in source_configs[source]:
        data = select_columns_safe(data, source_configs[source]['columns_to_select'])
    dataframe_collection["selected"][source] = data

# Block 4: Load & Create Table
spark.conf.set("spark.databricks.delta.schema.autoMerge.enabled", "true")
dataframe_collection["final"] = {}
for source, data in dataframe_collection["selected"].items():
    configs = source_configs[source]
    sink_path = f"{LKHS_PATH_DES}{configs['sink']['sink_directory']}"
    load_data_into_delta_table(data, sink_path, FULL_LOAD, configs["primary_col"]["primary_col_name"])
    create_lakehouse_table(f"{_SOURCE}_{configs['sink']['sink_directory']}", sink_path)
    step_results[f"Load_to_Delta_{source}"] = f"SUCCESS ({data.count()} records)"

# --- ADD THIS NEW BLOCK AT THE END ---
print("\n" + "="*50)
print("ETL STEP EXECUTION REPORT")
print("="*50)
for step, status in step_results.items():
    print(f"STEP: {step.ljust(30)} | STATUS: {status}")
print("="*50)

# Debug Check: If a step is missing from this list, you know exactly where the code stopped.

StatementMeta(, 9ec994a7-7ca8-47dc-8e17-08a4477051b0, 11, Finished, Available, Finished)

UnsupportedOperationException: [DELTA_MULTIPLE_SOURCE_ROW_MATCHING_TARGET_ROW_IN_MERGE] Cannot perform Merge as multiple source rows matched and attempted to modify the same
target row in the Delta table in possibly conflicting ways. By SQL semantics of Merge,
when multiple source rows match on the same target row, the result may be ambiguous
as it is unclear which source row should be used to update or delete the matching
target row. You can preprocess the source table to eliminate the possibility of
multiple matches. Please refer to
https://docs.delta.io/latest/delta-update.html#upsert-into-a-table-using-merge